# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install opencv-python-headless ultralytics

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [2]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [3]:
import cv2

video_capture = cv2.VideoCapture('YOLO_Video.mp4')


## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [4]:
height = video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = video_capture.get(cv2.CAP_PROP_FPS)
print ("height = ", height)
print ("width = ", width)
print ("fps = " , fps)

height =  1080.0
width =  1920.0
fps =  29.97002997002997


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [10]:
##output_path = 'output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')

width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

left_count = 0
right_count = 0

while video_capture.isOpened():

    ret, frame = video_capture.read()
    if not ret:
        break

    results = model(frame)

    for result in results[0].boxes:
        box = result.xyxy[0]
        x1, y1, x2, y2 = map(int, box)

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

height =  1080.0
width =  1920.0
fps =  29.97002997002997

In [ ]:
from ultralytics import YOLO

In [ ]:
!pip install ultralytics

from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow

video_path = "YOLO_Video.mp4"
cap = cv2.VideoCapture(video_path)

model = YOLO('yolov8n.pt')

while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model.track(frame, persist=True)

        annotated_frame = results[0].plot()

        cv2_imshow(annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

In [9]:
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

top_roi = (0, 0, 640, 100)
bottom_roi = (0, 300, 640, 100)
left_roi = (0, 100, 100, 200)
right_roi = (540, 100, 100, 200)

def count_vehicles(frame, roi):
    (x, y, w, h) = roi
    roi_frame = frame[y:y+h, x:x+w]
    results = model(roi_frame)

    objects = []
    for result in results:
        for det in result.boxes:
            if det.cls == 2:
                box = det.xyxy[0].tolist()  # [x1, y1, x2, y2]
                conf = det.conf[0]
                class_id = det.cls[0]
                objects.append((box, conf, class_id))

    return objects

def draw_info(frame, objects, roi, color=(0, 255, 0)):
    (x, y, w, h) = roi
    cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

    for (box, conf, class_id) in objects:
        (x1, y1, x2, y2) = box
        label = f'{model.names[class_id]}: {conf:.2f}'
        cv2.rectangle(frame, (int(x1 + x), int(y1 + y)), (int(x2 + x), int(y2 + y)), color, 2)
        cv2.putText(frame, label, (int(x1 + x), int(y1 + y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        center_x = int((x1 + x2) / 2 + x)
        center_y = int((y1 + y2) / 2 + y)
        cv2.putText(frame, f'Center: ({center_x}, {center_y})', (int(x1 + x), int(y2 + y) + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

video_path = 'input_video.mp4'
output_path = 'YOLO_Video1.mp4'
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

top_count = 0
bottom_count = 0
left_count = 0
right_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    top_objects = count_vehicles(frame, top_roi)
    bottom_objects = count_vehicles(frame, bottom_roi)
    left_objects = count_vehicles(frame, left_roi)
    right_objects = count_vehicles(frame, right_roi)

    draw_info(frame, top_objects, top_roi)
    draw_info(frame, bottom_objects, bottom_roi)
    draw_info(frame, left_objects, left_roi)
    draw_info(frame, right_objects, right_roi)

    top_count += len(top_objects)
    bottom_count += len(bottom_objects)
    left_count += len(left_objects)
    right_count += len(right_objects)

    cv2.putText(frame, f'Top: {top_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Bottom: {bottom_count}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Left: {left_count}', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Right: {right_count}', (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.putText(frame, 'Reshoof Alzweaid', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

In [30]:
import shutil

dataset_video_path = '/content/YOLO_Video.mp4'
video_path = '/content/YOLO_Video1.mp4'
shutil.copyfile(dataset_video_path, video_path)


best_model = YOLO('yolov8n.pt')


best_model.predict(source=video_path, save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/806) /content/YOLO_Video1.mp4: 384x640 1 car, 157.2ms
video 1/1 (frame 2/806) /content/YOLO_Video1.mp4: 384x640 1 car, 1 traffic light, 138.6ms
video 1/1 (frame 3/806) /content/YOLO_Video1.mp4: 384x640 2 cars, 1 traffic light, 149.9ms
video 1/1 (frame 4/806) /content/YOLO_Video1.mp4: 384x640 2 cars, 140.4ms
video 1/1 (frame 5/806) /content/YOLO_Video1.mp4: 384x640 3 cars, 2 traffic lights, 130.8ms
video 1/1 (frame 6/806) /content/YOL

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

## Save and Submit Your Work

Submit both the notebook and the output video